# Introduction to Linear Regression

In this tutorial, we'll discuss one of the foundational algorithms in machine learning: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables or labels*) by looking at the average temperature, rainfall, and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` using the training data, to make accurate predictions for new data. The _learned_ weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region. 

We'll _train_ our model by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*. 

## Imporint the libraries we need.

In [1]:
import torch

## Creating the data that we will use. 

In [2]:
# Input (temp, rainfall, humidity)
inputs = torch.tensor([[73, 67, 43], 
                       [91, 88, 64], 
                       [87, 134, 58], 
                       [102, 43, 37], 
                       [69, 96, 70],
                       [110, 20, 54],
                       [72, 93, 80],
                       [60, 70, 78]], dtype=torch.float32)

# Targets (apples, oranges)
targets = torch.tensor([[56, 70], 
                        [81, 101], 
                        [119, 133], 
                        [22, 37], 
                        [103, 119],
                        [16, 22],
                        [110, 97],
                        [34, 23]], dtype=torch.float32)

# We need those Tensors to be in that shape: column-vectors so the can be compatable with Pytorch

## Linear regression model from scratch.

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [4]:
weights = torch.randn(3, 2, requires_grad=True, dtype=torch.float32)
biases = torch.randn(2, requires_grad=True, dtype=torch.float32)

tensor([[-1.6740, -0.2723],
        [ 2.2429, -0.5075],
        [-1.2366,  0.0303]], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [14]:
def model(inputs):
    return inputs @ weights + biases # inputs: shape=(8,3) and weights: shape=(3,2)

`@` represents matrix multiplication in PyTorch.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [15]:
# Generating predictions
preds = model(inputs)
print("Predictions:")
print(preds, '\n')
print("Desire Outputs:")
print(targets)

Predictions:
tensor([[ 64.4213,  64.4509],
        [ 80.6047,  86.8863],
        [167.8715, 114.9514],
        [ 15.6859,  48.9545],
        [ 98.4472,  92.3810],
        [-39.8707,  40.1278],
        [ 85.0638,  94.0099],
        [ 49.9124,  77.1406]], grad_fn=<AddBackward0>) 

Desire Outputs:
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 16.,  22.],
        [110.,  97.],
        [ 34.,  23.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to *just work*.


## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [16]:
def mse_loss(preds, tergets):
    differ = preds - targets
    return torch.sum(differ * differ) / differ.numel() # Same can be written as 'torch.mean(differ * differ)

# Compute loss
loss = mse_loss(preds, targets)

Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [17]:
# Calculate gradients
loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [18]:
# Seeing the gradients of the loss with respect of the weigths
print(weights.grad)

tensor([[-389.2505,  143.7733],
        [ 505.8805, -274.9218],
        [-143.9613,  169.1169]])


## Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

* **increasing** the weight element's value slightly will **increase** the loss
* **decreasing** the weight element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

* **increasing** the weight element's value slightly will **decrease** the loss
* **decreasing** the weight element's value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [19]:
with torch.no_grad():
    weights -= weights.grad * 1e-4
    biases -= biases.grad * 1e-4

We multiply the gradients with a very small number (`10^-4` in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the *learning rate* of the algorithm. 

We use `torch.no_grad` to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [20]:
for _ in range(100):
    preds = model(inputs)
    loss = mse_loss(preds, targets)
    loss.backward()
    with torch.no_grad():
        weights -= weights.grad * 1e-5
        biases -= biases.grad * 1e-5
        weights.grad.zero_()
        biases.grad.zero_()
    
# Generating predictions
preds = model(inputs)
print("Predictions:")
print(preds, '\n')
print("Desire Outputs:")
print(targets)

Predictions:
tensor([[ 64.4480,  65.4033],
        [ 80.5259,  87.2572],
        [148.8806, 126.3986],
        [ 31.0115,  43.4936],
        [ 90.7523,  95.2117],
        [-11.4953,  26.2656],
        [ 80.7209,  94.2652],
        [ 50.7513,  73.8425]], grad_fn=<AddBackward0>) 

Desire Outputs:
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 16.,  22.],
        [110.,  97.],
        [ 34.,  23.]])


## Summaring the Feed Forward Neural Network.


In [21]:
import torch


# Input (temp, rainfall, humidity)
inputs = torch.tensor([[73, 67, 43], 
                       [91, 88, 64], 
                       [87, 134, 58], 
                       [102, 43, 37], 
                       [69, 96, 70],
                       [110, 20, 54],
                       [72, 93, 80],
                       [60, 70, 78]], dtype=torch.float32)

# Targets (apples, oranges)
targets = torch.tensor([[56, 70], 
                        [81, 101], 
                        [119, 133], 
                        [22, 37], 
                        [103, 119],
                        [16, 22],
                        [110, 97],
                        [34, 23]], dtype=torch.float32)

# Creating a class for the Neural Network.
class Linear_Regression:
    def __init__(self, input_layers, target_layers):
        # Initialize the weights and the biases randomly.
        self.weights = torch.randn(input_layers, target_layers, requires_grad=True, dtype=torch.float32)
        self.biases = torch.randn(target_layers, requires_grad=True, dtype=torch.float32)

    def __call__(self, inputs):
        # Creating the model (returns the predictions)
        return (inputs @ self.weights) + self.biases

    def mse_loss(self, preds, targets):
        # Calculating the Mean Squared Error, loss of our model.
        diff = preds - targets
        return torch.sum(diff * diff) / diff.numel()

    def fit(self, epochs, learning_rate, inputs, targets):
        # Training our model for a specified number of epochs (iteratrions).
        for _ in range(epochs):
            preds = self(inputs)
            loss = self.mse_loss(preds, targets)
            loss.backward()
            with torch.no_grad():
                # Update the weights and biases with respect of the loss.
                self.weights -= (self.weights.grad * learning_rate)
                self.biases -= (self.biases.grad * learning_rate)
                # Reseating the grads to be ready for the next epoch.
                self.weights.grad.zero_()
                self.biases.grad.zero_()


# Creating the model object
model = Linear_Regression(3, 2)

In [22]:
# Training the model for 100 epochs.
model.fit(100, 1e-5, inputs, targets)
loss = model.mse_loss(model(inputs), targets)
pred = model(torch.tensor([90, 60, 65], dtype=torch.float32))
print(pred, loss) 

tensor([52.5285, 53.6255], grad_fn=<AddBackward0>) tensor(585.7444, grad_fn=<DivBackward0>)


Gradient Descent with little more mathematics

https://storage.googleapis.com/supplemental_media/udacityu/315142919/Gradient%20Descent.pdf